# Diabetes Classification Dataset

This data is from the Behavioral Risk Factor Surveillance System (BRFSS) which is a health-related telephone survey that is collected annually by the CDC. The goal of this project is to train classification machine learning models to compare model performance based on model type and dataset characteristics.  

The original data is the linked Dataset from the University of California Irvine ML Repository. The linked data opens a kaggle dataset which was obtained from the original CDC posted kaggle dataset. 

The CSV file of interest to compare model performance:
1. diabetes _ 012 _ health _ indicators _ BRFSS2015.csv is a clean dataset of 253,680 survey responses to the CDC's BRFSS2015. The target variable Diabetes_012 has 3 classes. 0 is for no diabetes or only during pregnancy, 1 is for prediabetes, and 2 is for diabetes. There is class imbalance in this dataset. This dataset has 21 feature variables

https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators

### Data Imports, package imports and dataframe setup

In [5]:
# Import all necessary libraries for the project
import numpy as np 
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

In [6]:
# Read the data into a DataFrame in pandas
data = pd.read_csv('diabetes_prediction/diabetes_012_health_indicators_BRFSS2015.csv')

# Exploratory Data Analysis

In [7]:
# Compare dataframe sizes 
print(f"Shape of dataset:{data.shape}")
# All column names
print(data.columns)

Shape of dataset:(253680, 22)
Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')


In [8]:
# Data types for all columns 
print(data.dtypes)

# Print out the first five rows of the balanced dataset 
data.head()

Diabetes_012            float64
HighBP                  float64
HighChol                float64
CholCheck               float64
BMI                     float64
Smoker                  float64
Stroke                  float64
HeartDiseaseorAttack    float64
PhysActivity            float64
Fruits                  float64
Veggies                 float64
HvyAlcoholConsump       float64
AnyHealthcare           float64
NoDocbcCost             float64
GenHlth                 float64
MentHlth                float64
PhysHlth                float64
DiffWalk                float64
Sex                     float64
Age                     float64
Education               float64
Income                  float64
dtype: object


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [9]:
# Check for columns with NA values
print(data.isna().sum())

Diabetes_012            0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64


In [50]:
print("Percent of diabetes/prediabetes in balanced set:", data['Diabetes_binary'].sum() / len(data))

Percent of diabetes/prediabetes in balanced set: 0.5
Percent of diabetes/prediabetes in unbalanced set: 0.13933301797540207


## Determine which columns we can remove before modeling 

In [81]:
correlations = data.corrwith(data['Diabetes_binary'])
print(correlations.sort_values(ascending=False))

Diabetes_binary         1.000000
GenHlth                 0.407612
HighBP                  0.381516
BMI                     0.293373
HighChol                0.289213
Age                     0.278738
DiffWalk                0.272646
PhysHlth                0.213081
HeartDiseaseorAttack    0.211523
Stroke                  0.125427
CholCheck               0.115382
MentHlth                0.087029
Smoker                  0.085999
Sex                     0.044413
NoDocbcCost             0.040977
AnyHealthcare           0.023191
Fruits                 -0.054077
Veggies                -0.079293
HvyAlcoholConsump      -0.094853
PhysActivity           -0.158666
Education              -0.170481
Income                 -0.224449
dtype: float64


# Data prep for modeling

In [66]:
random_state = 12

# Balanced Model variables
X = data.drop(["Diabetes_binary", "GenHlth"], axis = 1)
y = data['Diabetes_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=random_state)

In [67]:
clf = LogisticRegression(random_state=random_state, max_iter=1000).fit(X_train, y_train)

y_pred_b = clf.predict(X_test)

accuracy_score(y_pred=y_pred_b, y_true=y_test)